# Module 11: Poke the Dots Version 5

## Solution Issues in Poke the Dots Version 4

There's only one solution issue left from version four since nothing happens when the dots collide. For this version of poke the dots, we are going to detect when the dots collide and change the game behavior accordingly. We have already implemented some collision detection with regard to the edges of the window, which equations deal with one non-moving object (the window and one moving object). For this version, we have to investigate how to handle collision between two moving objects. 

## Observe Poke the Dots Version 5

In the final version of Poke the Dots nothing seems different from the previous version. The dots are red and blue, they move at the same speed as version four, and they bounce when they hit the edge of the window. The scoreboard is still white and tallies the number of seconds since the game was started. When we press the mouse button the dots teleport to random locations around the window. However, when the dots collide, the dots immediately stop and a **Game Over** message appears in a corner of the screen. Notice that the text has the color of the small dot and the background has the color of the big dot as in the example below.

<img src="https://raw.githubusercontent.com/rogergranada/MOOCs/master/Coursera/University%20of%20Alberta/Problem%20Solving%2C%20Python%20Programming%2C%20and%20Video%20Games/Week%2009/images/poke_the_dots.gif" width="40%" align="center"/>

## Describe Poke the Dots Version 5

In this description, we add the information when the dots collide and the GAME OVER message. The complete description is below.

<img src="images/description_version_5.svg" width="40%" align="center"/>

## Create Test Plan for Poke the Dots Version 5

We must test the GAME OVER behaviour when the dots collide, including the GAME OVER message. The dots do not move once they collide, which means you must test what happens when the player clicks inside the window after the dots have collided. The complete test plan for this version is described below.

<img src="images/testplan_version_5.svg" width="80%" align="center"/>

## Create Algorithm for Poke the Dots Version 5

In the final Poke algorithm, we must add steps for the only remaining game feature, `Dot` collision. You must check for `Dot` collision every frame. Dot collision is not the result of a player event, it is a result of moving game objects. Add a new attribute to the `Game` class to keep track of whether the game should continue after the Dots collide. Use this attribute to select the appropriate actions when drawing and updating the game. For example, when game is over, both draw and update must treat some game objects differently. The complete algorithm is as follows:

<img src="images/algorithm_version_5.svg" width="100%" align="center"/>

## Program Poke the Dots Version 5

We finish the version 5 of Poke the Dots, adding the GAME OVER feature when the balls collide. The complete algorithm is presented below.

In [5]:
# Poke the Dots version 5
# This is a graphical game where two dots move on
# the screen, bouncing off the edges.

from uagame import Window
from pygame import time
from pygame.time import Clock
from pygame.event import get as get_events
from pygame import QUIT, Color, MOUSEBUTTONUP
from pygame.draw import circle as draw_circle
import random
from math import sqrt

def main():
    # Create game
    game = Game()
    game.play()

class Game:
    # An object in this class represents a complete game
    # - window
    # - frame_rate
    # - close_selected
    # - clock
    # - small_dot
    # - big_dot
    def __init__(self):
        self._window = Window('Poke the Dots', 500, 400)
        self._clock = Clock()
        self._frame_rate = 90
        self._close_selected = False
        self._small_dot = Dot('red', [200, 100], 20, [1, 2], self._window)
        self._small_dot.randomize()
        self._big_dot = Dot('blue', [200, 100], 40, [2, 1], self._window)
        self._big_dot.randomize()
        self._score = 0
        self._continue = True
        self._adjust_window()
    
    def _adjust_window(self):
        # set parameters to the window
        self._window.set_font_name('ariel')
        self._window.set_font_size(64)
        self._window.set_font_color('white')
        self._window.set_bg_color('black')

    def _draw(self):
        # draw small and big dots on the screen and update the window
        self._window.clear()
        self._draw_score()
        self._small_dot.draw()
        self._big_dot.draw()
        if not self._continue:
            self._draw_game_over()
        self._window.update()

    def _draw_score(self):
        # draw the score on the screen
        self._window.draw_string('Score: '+str(self._score), 0, 0)

    def _draw_game_over(self):
        # draw GAME OVER when the balls collide
        small_color = self._small_dot.get_color()
        big_color = self._big_dot.get_color()
        font_color = self._window.get_font_color()
        bg_color = self._window.get_bg_color()
        self._window.set_font_color(small_color)
        self._window.set_bg_color(big_color)
        height = self._window.get_height() - self._window.get_font_height()
        self._window.draw_string('GAME OVER', 0, height)
        self._window.set_font_color(font_color)
        self._window.set_bg_color(bg_color)

    def _handle_events(self):
        # handle the exit game event when the player clicks [X]
        event_list = get_events()
        for event in event_list:
            if event.type == QUIT:
                self._close_selected = True
            elif event.type == MOUSEBUTTONUP and self._continue:
                self._small_dot.randomize()
                self._big_dot.randomize()

    def play(self):
        # run the game while the player do not select to close the window
        while not self._close_selected:
            self._handle_events()
            self._draw()
            self._update()
        self._window.close()

    def _update(self):
        # control the frame rate and update the movement of the dots
        if self._continue:
            self._small_dot.move()
            self._big_dot.move()
            self._score = time.get_ticks() // 1000
        self._clock.tick(self._frame_rate)
        if self._small_dot.intersects(self._big_dot):
            self._continue = False
            

class Dot:
    # window is the Window to display in
    # - color is the string representing the color of the small dot
    # - center is a list representing the X and Y positions of the center of the small dot
    # - radius is an int representing the radius of the small dot
    # - velocity is a list representing the velocity in X and Y of the small dot
    # - clock is the Clock object representing time
    def __init__(self, color, center, radius, velocity, window):
        self._color = color
        self._center = center
        self._radius = radius
        self._velocity = velocity
        self._window = window

    def get_color(self):
        # return the color of the font
        return self._color

    def get_center(self):
        # return the center of the dot
        return self._center

    def get_radius(self):
        # return the radius of the dot
        return self._radius

    def draw(self):
        # draw the dot on the screen
        surface = self._window.get_surface()
        color = Color(self._color)
        draw_circle(surface, color, self._center, self._radius)

    def move(self):
        # update the movement of the dot in each axis X and Y
        size = [self._window.get_width(), self._window.get_height()]
        for index in range(0, 2):
            # 0 horizontal, 1 vertical
            self._center[index] = self._center[index] + self._velocity[index]
            if self._center[index] + self._radius >= size[index] or self._center[index] - self._radius <= 0:
                self._velocity[index] = - self._velocity[index]

    def randomize(self):
        # randomize the position of Dot
        size = [self._window.get_width(), self._window.get_height()]
        for index in range(0, 2):
            self._center[index] = random.randint(0+self._radius, size[index])

    def intersects(self, dot):
        # check whether two dots intersect
        distance = 0
        for index in range(0, 2):
            distance += (self._center[index] - dot.get_center()[index])**2
        sum_radius = self._radius + dot.get_radius()
        if sqrt(distance) <= sum_radius:
            return True
        return False

main()

## Solution Issues in Poke the Dots Version 5

Since this is the last version of Poke the Dots, we only have one more task in the game creation process diagram to finish. Let's explore some of the potential solution issues that could be solved to improve Poke the Dots. Let's investigate some of the improvements that can be made to Poke the Dots. 

**1.** Poke the Dots is not a visually interesting game. One small modification would be making an addition to the mouse up event handler. Every time you click the mouse button, you could randomly change the colors of the dots.<br> 
**2.** Poke the Dots can be boring since it isn't very challenging. You could make the game more interesting by making it more challenging. Here are three ways to increase the difficulty to hopefully increase enjoyment. 
- Every time you press the mouse button, you could increase the speed of the dots. This would force the player to be very careful about when they press the mouse button, as pressing too often will make the dots very fast. 
- Although this game is called Poke the Dots, you can click anywhere in the window to teleport the dots. A more accurate and challenging rendition of the game could be created by modifying your mouse up event handler to only teleport if the mouse cursor is inside a dot when clicked. 
- Another way to increase the difficulty of Poke the Dots is to add a new dot whenever the mouse cursor is clicked outside of the dot. This idea works especially well if you include the feature of only teleporting the dots when the mouse is clicked inside the dot. Instead of doing nothing if the player clicks outside of the dot, you could add a punishment that every missed timed click adds a new dot with a random color and size to the screen. You might want to make the window a bit larger if you try this improvement.

**3.** Finally, we could make the game easier to replay, since currently, you must close the window and then restart the game. Alternately, you can add an additional event handler that checks if the game is over and a replay key is pressed. If so, the game can be restarted by setting continue game to true. However, you must figure out a way to reset the score to zero each time you do a restart. 

These are only a few of the many improvements that could be made to Poke the Dots.

---
# Quiz

# Understand Poke the Dots Version 5

**1. Which of these features are present in Poke the Dots Version 5, but not present in Poke the Dots Version 4?**

&#9744; The score increments once per second, unless the game has ended.<br>
&#9744; There is a scoreboard in the top left corner of the window.<br>
&#9744; When a dot hits an edge of the window it bounces.<br>
&#9745; The font colour of the "GAME OVER" message is the colour of the small dot on a background that is the colour of the big dot.<br>
&#9745; When the game ends a "GAME OVER" message is displayed.<br>
&#9744; Clicking the close icon at any time closes the game window.<br>
&#9744; During the game, when the player clicks the left mouse button with the cursor inside the window, both dots teleport to another location in the window.<br>
&#9745; When the dots collide, the game ends.<br>
&#9745; When the game ends the dots stop moving.<br>
&#9744; There are two dots that move in the window.<br>
&#9744; One dot is red and one dot is blue.

## Program Poke the Dots Version 5

Finish programming Poke the Dots Version 5. Select all of the functional test blocks that your code passes. To pass a test block, the answer must be "yes" to all questions in that block.


&#9745; **1. Start the program**
- Does the game open a window?
- Does it have title Poke the Dots?
- Does it have a black background?
- Does it have aspect ratio 5:4?

&#9745; **2 Does the game display a small dot?**
- Is it red?
- Does it start in the top left corner?
- Does it start moving down and to the right?
- Does it move in a straight line?
- Does it move at a constant speed?
- Does it move twice as fast in the vertical direction as the horizontal direction?
- Does it bounce off the window edges?

&#9745; **3 Does the game display a big dot?**
- Is it blue?
- Does it start in the top left corner?
- Does it start moving down and to the right?
- Does it move in a straight line?
- Does it move at a constant speed?
- Does it move twice as fast in the horizontal direction as the vertical direction?
- Does it bounce off the window edges?

&#9745; **4 Does the game display a scoreboard?**
- Does it indicate the time since the game started?
- Does it start at 0?
- Is it in the top left corner of the window?
- Does it use large font size? Is it white on black?

&#9745; **5 Click the mouse inside the window**
- Do the dots teleport to random locations?
- Do the dot velocities remain the same?
- Do the dot trajectories remain the same?

&#9745; **6 Click the mouse inside the window**
- Do the dots teleport to random locations?
- Do the dot velocities remain the same?
- Do the dot trajectories remain the same?

&#9745; **7 Wait for the dots to collide**
- Do the dots stop moving?

&#9745; **8 Does the score stop incrementing?**

&#9745; **9 Is a game over message displayed?**
- Is it in the bottom left corner of the window?
- Does it use large font size?
- Does it use small dot colour on big dot colour?

&#9745; **10 Click the mouse inside the window**
- Does nothing happen?

&#9745; **11 Click the close icon**
- Does the game close the window?

&#9745; **12 Does the program end?**

&#9745; **13 Restart the program**
- Does the game display a small dot?
- Does it start in a random location?

&#9745; **14 Does the game display a big dot?**
- Does it start in a random location?

&#9745; **15 Click the close icon**
- Does the game close the window?

&#9745; **16 Does the program end?**